# marine controlled-source EM inversion -- _with automatic grid_

- use of `emg3d` + `SimPEG`

based on `simpeg-emg3d-inversion.ipynb`

This example uses a different computational mesh than the inversion mesh; but the same for all sources and all frequencies. The main point is that the computational mesh has a much larger extent than the inversion mesh.

Using the up-to-date (2022-03-11) branch:

- SimPEG: https://github.com/simpeg/simpeg/tree/emg3d-solver

and `emg3d>v1.4.0` (latest on `main` or any newer release).

In [6]:
import numpy as np
import discretize
import matplotlib.pyplot as plt
import simpeg.electromagnetics.frequency_domain as FDEM
%matplotlib widget

ImportError: cannot import name 'LocationVector' from 'simpeg.props' (/home/dtr/Codes/simpeg/simpeg/props.py)

## Create a model

In [ ]:
import emg3d
import numpy as np
frequency = 2.
mesh_tmp = emg3d.construct_mesh(
    center=(0, 0, 0),    # Center of wanted grid
    frequency=frequency,        # Frequency we will use the grid for
    properties=2,        # Reference resistivity
    domain=[-2000, 2000],  # Domain in which we want precise results
    center_on_edge=True,
)

In [ ]:
hx = mesh_tmp.h[0]
hz = mesh_tmp.h[2]
hy = np.r_[mesh_tmp.h[1][:8], mesh_tmp.h[1][8], mesh_tmp.h[1][-8:]]

In [ ]:
mesh = discretize.TensorMesh([hx, hy, hz], origin=[mesh_tmp.x0[0], -hy.sum()/2, mesh_tmp.origin[2]])

In [ ]:
from geoana import em
from matplotlib.colors import LogNorm
em.fdem.skin_depth(1, 1) * 4

In [ ]:
inds_air = mesh.cell_centers[:,2]>0.
inds_seafloor = mesh.cell_centers[:,2]<-1000.
inds_hydrocarbon = (
    np.logical_and(mesh.cell_centers[:,2]<-1500, mesh.cell_centers[:,2]>-1700) 
    & np.logical_and(mesh.cell_centers[:,0]>-1000, mesh.cell_centers[:,0]<1000)
    & np.logical_and(mesh.cell_centers[:,1]>-1000, mesh.cell_centers[:,1]<1000)
)

sigma = np.ones(mesh.n_cells) * 1./0.33
sigma[inds_air] = 1./1e8
sigma[inds_seafloor] = 1./1.
sigma[inds_hydrocarbon] = 1./100
fig, ax = plt.subplots(1,1, figsize=(8, 4))
mesh.plot_slice(
    1./sigma, grid=True, normal='Y', 
#     grid_opts={'color':'grey', 'linewidth':1},
    pcolor_opts={'cmap':'Spectral_r', 'norm':LogNorm(vmin=0.33, vmax=100)}, 
    ax=ax,
)
ax.set_aspect(1)
ax.set_ylim(mesh.nodes_z.min(), 0)

In [ ]:
inds_active = mesh.cell_centers[:,2]<-1000.

## Create a survey

In [ ]:
mesh.cell_centers_z

In [ ]:
from simpeg import utils
# there is funcy behavior ... had to use this way ???
rec_x = mesh.cell_centers_x[8:-8]
rec_y =  np.r_[-83.88202017]
rec_z = np.array([-8.38820202e+02])
xyz_rx = utils.ndgrid(rec_x, rec_y, rec_z)
src_x = mesh.cell_centers_x[8:-8][::3]
src_y = np.r_[-83.88202017]
src_z = np.array([-1000])
xyz_src = utils.ndgrid(src_x, src_y, src_z)

In [ ]:
mesh.plot_slice(
    1./sigma, grid=True, normal='Z', 
    ind=10,
    pcolor_opts={'cmap':'Spectral_r', 'norm':LogNorm(vmin=1, vmax=100)}, 
)
plt.plot(xyz_rx[:,0], xyz_rx[:,1], 'r.')
plt.plot(xyz_src[:,0], xyz_src[:,1], 'w*')
plt.gca().set_aspect(1)
plt.xlim(-5000, 5000)
plt.ylim(-5000, 5000)

In [ ]:
rx_list = [
    FDEM.receivers.PointElectricField(
        orientation='x', component="complex", locations=xyz_rx),
]

# vector = np.real(sfield.field/-sfield.smu0)
src_list = []
for ii in range(xyz_src.shape[0]):
    src = FDEM.sources.ElectricDipole(rx_list, location=xyz_src[ii,:], freq=frequency, azimuth=0, elevation=0)
    src_list.append(src)
survey = FDEM.Survey(src_list)

## Create simulation

In [ ]:
from SimPEG.electromagnetics.frequency_domain import Simulation3DEMG3D
from SimPEG import maps
from pymatsolver import Pardiso

active_map = maps.InjectActiveCells(mesh, inds_active, sigma[~inds_active])
nP = int(inds_active.sum())
conductivity_map = active_map * maps.ExpMap(nP=nP)
# conductivity_map = maps.IdentityMap(mesh)

In [ ]:
# Define the Simulation
simulation = Simulation3DEMG3D(
        mesh,
        survey=survey,
        sigmaMap=conductivity_map,
        Solver=Pardiso,
        verbose=False,
        simulation_opts={
            'max_workers': 6, 
            'gridding': 'single',
            'model': emg3d.Model(mesh, sigma, mapping='Conductivity'),
            'gridding_opts': {
                'seasurface': 0,
                'center': (0, 0, -1000),
                'vector': (mesh.vectorNx[8:-8], mesh.vectorNy[2:-2], mesh.vectorNz[8:-20]),
                'domain': ([-1600, 1600], [-1600, 1600], [-2000, 0]),
                'min_width_limits': (200, 400, 100),
                'max_buffer': 20000,
                'lambda_from_center': True,
            },
#            'solver_opts': {
#                'maxit': 1,     # To speed up for testing
#                'plain': True   # " "  (fails faster)
#            },
#            'verb': -1,         # To not raise convergence warnings
        },
)

m_true =  np.log(sigma[inds_active])
# m_true =  np.ones(mesh.nC)
m0 =  np.ones(inds_active.sum()) * np.log(1.)

m2 = simulation.emg3d_sim.get_model('TxED-1', 'f-1')
m2.grid

In [ ]:
from matplotlib.colors import LogNorm
m2.grid.plot_3d_slicer(1/m2.property_x, xslice=12000, yslice=7000,
                       zlim=[-2500, 500],
                       pcolor_opts={'norm': LogNorm(vmin=0.3, vmax=200)})

In [ ]:
f = simulation.fields(m_true)
d_true = simulation.dpred(m_true, f=f)

In [ ]:
d_0 = simulation.dpred(m0)

In [ ]:
relative_error = 0.01
noise_floor = 1e-14
standard_deviation = np.sqrt(abs(relative_error*d_true)**2 + (noise_floor)**2)

In [ ]:
residual = (d_true - d_0)/standard_deviation

In [ ]:
np.vdot(residual, residual).real

In [ ]:
plt.semilogy(abs(residual), label='dobs')
# plt.semilogy(standard_deviation,label='std')

In [ ]:
plt.hist(np.log10(abs(d_true)))

In [ ]:
plt.semilogy(abs(d_true), '.', label='pred')


In [ ]:
from SimPEG import (
    maps,
    data,
    data_misfit,
    regularization,
    optimization,
    inverse_problem,
    inversion,
    directives,
    utils,
)

In [ ]:
em_data = data.ComplexData(survey, dobs=d_true, standard_deviation=standard_deviation)
dmis = data_misfit.L2DataMisfit(data=em_data, simulation=simulation)

In [ ]:
from SimPEG import tests
adjoint_tol = 1e-10
def test_misfit():
    passed = tests.checkDerivative(
        lambda m: (simulation.dpred(m), lambda mx: simulation.Jvec(m0, mx, f=f)),
        m0,
        plotIt=False,
        num=3,
    )

def test_adjoint():
    # Adjoint Test
    f = simulation.fields(m=m0)
    # u = np.random.rand(.mesh.nC * .survey.nSrc)
    v = np.random.rand(inds_active.sum())
#     v = np.random.rand(mesh.nC)
    w = np.random.rand(simulation.survey.nD)
    wtJv = np.vdot(w, simulation.Jvec(m0, v, f=f)).real
    vtJtw = np.vdot(v, simulation.Jtvec(m0, w, f=f))
    passed = np.abs(wtJv - vtJtw) < adjoint_tol
    print("Adjoint Test", np.abs(wtJv - vtJtw), passed)
    print(wtJv, vtJtw)
    
def test_dataObj():
    passed = tests.checkDerivative(
        lambda m: [dmis(m), dmis.deriv(m)], m0, plotIt=False, num=2
    )    

In [ ]:
test_dataObj()

In [ ]:
test_misfit()

In [ ]:
test_adjoint()

In [ ]:
from SimPEG import utils

In [ ]:
%%time
# Define the regularization (model objective function)
reg = regularization.Tikhonov(
    mesh,
    indActive=inds_active,
    mref=m0,
    alpha_s=1e-8,
    alpha_x=1,
    alpha_y=10,
    alpha_z=1
)

opt = optimization.InexactGaussNewton(
    maxIter=20, maxIterLS=20, maxIterCG=20, tolCG=1e-3
)

inv_prob = inverse_problem.BaseInvProblem(dmis, reg, opt)
starting_beta = directives.BetaEstimate_ByEig(beta0_ratio=1)
save = directives.SaveOutputDictEveryIteration()

beta_schedule = directives.BetaSchedule(coolingFactor=2, coolingRate=1)
target_misfit = directives.TargetMisfit(chifact=1)

directives_list = [
    starting_beta,
    beta_schedule,
    target_misfit,
    save
]
em_inversion = inversion.BaseInversion(inv_prob, directiveList=directives_list)

# Run inversion
recovered_conductivity_model = em_inversion.run(m0)

In [ ]:
target_misfit.target

In [ ]:
plt.figure(figsize=(10, 4))
#iteration = 16
iteration = len(save.outDict.keys())
plt.semilogy(abs(em_data.dobs), '-', label='Observed')
plt.semilogy(abs(save.outDict[iteration]['dpred']), '.', label='Predicted')
plt.legend()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(8, 4))

sigm_est = conductivity_map * save.outDict[iteration]['m']
sigmas = [sigm_est, sigma]
titles = ["Estimated", "True"]
for ii, ax in enumerate(axs):
    out = mesh.plot_slice(
        1./sigmas[ii], grid=False, normal='Y', 
        pcolor_opts={'cmap':'Spectral_r', 'norm':LogNorm(vmin=0.33, vmax=100)}, 
        ax=ax,
    )
    ax.set_aspect(1)
    ax.set_ylim(-4000, 0)
    ax.set_xlim(-2000, 2000)
    if ii == 1:
        ax.set_yticks([])
    ax.set_title(titles[ii])
    cb = plt.colorbar(out[0], ax=ax, fraction=0.03, orientation='horizontal')
    cb.set_label("Resistivity ($\Omega$ m)")

In [ ]:
import SimPEG
SimPEG.Report('emg3d')